First conceptual idea of splitting each row of the data set by hour for the predictive analytics models (in order to later re-aggregate the data by hour more easily)

In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
rf_data = pd.read_csv("../cleanData/cleanChargingDataFull.csv")

In [3]:
rf_data["connectionTime"] = pd.to_datetime(rf_data["connectionTime"])
rf_data["disconnectTime"] = pd.to_datetime(rf_data["disconnectTime"])


C:\Users\tim7l\AppData\Local\Temp\ipykernel_8232\357342468.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  rf_data["connectionTime"] = pd.to_datetime(rf_data["connectionTime"])
C:\Users\tim7l\AppData\Local\Temp\ipykernel_8232\357342468.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  rf_data["disconnectTime"] = pd.to_datetime(rf_data["disconnectTime"])


In [4]:
rf_data = rf_data.drop_duplicates(subset=["connectionTime", "spaceID"], ignore_index=True)
rf_data

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID_x,parkDuration,WhPerMile,kWhRequested,milesRequested,minutesAvailable,requestedDeparture,temperature,cloud_cover,precipitation
0,0,51323,2020-11-18 15:36:26-08:00,2020-11-18 16:02:37-08:00,NaN,4.816,2,11900388,2-39-81-4550,7132.0,0 days 00:26:11,274.0,8.22,30.0,480.0,"Thu, 19 Nov 2020 07:36:26 GMT",13.15,27.46,0.0
1,1,51324,2020-11-18 16:35:54-08:00,2020-11-18 17:31:08-08:00,NaN,10.027,2,11900388,2-39-81-4550,4903.0,0 days 00:55:14,258.0,51.60,200.0,576.0,"Thu, 19 Nov 2020 10:11:54 GMT",13.15,27.46,0.0
2,2,51325,2020-11-18 17:34:02-08:00,2020-11-18 18:45:14-08:00,NaN,24.486,2,11900388,2-39-81-4550,4903.0,0 days 01:11:12,258.0,51.60,200.0,576.0,"Thu, 19 Nov 2020 11:10:02 GMT",13.15,27.46,0.0
3,3,51328,2020-11-18 19:52:00-08:00,2020-11-18 20:00:50-08:00,NaN,4.788,2,11900388,2-39-81-4550,1085.0,0 days 00:08:50,283.0,56.60,200.0,589.0,"Thu, 19 Nov 2020 13:41:00 GMT",13.15,27.46,0.0
4,4,51329,2020-11-18 20:24:11-08:00,2020-11-18 21:07:15-08:00,NaN,30.849,2,11900388,2-39-81-4550,9284.0,0 days 00:43:04,400.0,40.00,100.0,30.0,"Thu, 19 Nov 2020 04:54:11 GMT",13.15,27.46,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65004,65004,24487,2019-05-13 12:43:12-07:00,2019-05-13 14:54:33-07:00,2019-05-13 13:46:37-07:00,2.308,2,CA-513,2-39-139-567,560.0,0 days 02:11:21,273.0,5.46,20.0,133.0,"Mon, 13 May 2019 21:56:12 GMT",17.19,25.71,0.0
65005,65005,25429,2019-06-13 09:53:42-07:00,2019-06-13 10:35:12-07:00,2019-06-13 10:24:57-07:00,0.908,2,CA-513,2-39-139-567,NaN,0 days 00:41:30,NaN,NaN,NaN,NaN,NaN,18.27,26.95,0.0
65006,65006,28030,2019-09-16 06:55:08-07:00,2019-09-16 08:34:27-07:00,2019-09-16 07:25:44-07:00,0.893,2,CA-513,2-39-139-567,NaN,0 days 01:39:19,NaN,NaN,NaN,NaN,NaN,22.00,29.38,0.0
65007,65007,29515,2019-11-06 07:28:18-08:00,2019-11-06 08:41:56-08:00,2019-11-06 07:58:42-08:00,0.900,2,CA-513,2-39-139-567,NaN,0 days 01:13:38,NaN,NaN,NaN,NaN,NaN,15.88,33.56,0.0


In [5]:
rf1_data = rf_data[rf_data["siteID"] == 1]
rf2_data = rf_data[rf_data["siteID"] == 2]

**Disaggregate sessions by hour**

In [6]:
# Function to disaggregate sessions by hour
def disaggregate_session(row):
    # Generate a list of hourly intervals
    current = row['connectionTime']
    end = row['disconnectTime']
    rows = []
    while current < end:
        # Ensure that intervals stay within the session's bounds
        if current == current.ceil("h"):
            next_hour = min(current + timedelta(hours=1), end)
        else:
            next_hour = min(current.ceil("h"), end)
        rows.append({'id': row['id'], 
                     'connectionTime': row['connectionTime'], 
                     'disconnectTime': row['disconnectTime'],
                     'inHourStartTime': current,
                     'inHourEndTime': next_hour,
                     'minutesInHour' : ((next_hour - current).seconds) / 60})
        current = next_hour
    return rows

In [7]:
dis_rf1_data = rf1_data.apply(disaggregate_session, axis=1)
dis_rf2_data = rf2_data.apply(disaggregate_session, axis=1)

**Re-aggregate data & fill new dataframe**

In [8]:
def agg_rows(p_df, first_year, last_year):
    # Generate a range of hourly timestamps for the entire year
    start_time = f'{first_year}-04-24 00:00:00+00:00'
    end_time = f'{last_year}-09-14 23:00:00+00:00'
    hourly_range = pd.date_range(start=start_time, end=end_time, freq='h')

    # Create a DataFrame with the timestamps
    this_df = pd.DataFrame(hourly_range, columns=['datetime'], index=hourly_range)

    # Add additional columns if needed (e.g., placeholder values)
    this_df['hour'] = this_df['datetime'].map(lambda x: x.hour)
    this_df['weekday'] = this_df['datetime'].map(lambda x: x.weekday())
    this_df['dayOfMonth'] = this_df['datetime'].map(lambda x: x.day)
    this_df['month'] = this_df['datetime'].map(lambda x: x.month)
    this_df['year'] = this_df['datetime'].map(lambda x: x.year)
    this_df['total number of sessions'] = 0
    this_df['total minutes of parking'] = 0.0
    #this_df['total kWh requested'] = 0

    # Fill dataframe
    for session in p_df:
        for cur_row in session:
            # OPtimize method by splitting data set that is searched
            cur_connectionTime = cur_row["inHourStartTime"]
            cur_minutesInHour = cur_row["minutesInHour"]
            cur_floor_connectionTime = cur_connectionTime.floor('h')

            this_df.loc[cur_floor_connectionTime, "total number of sessions"] += 1
            this_df.loc[cur_floor_connectionTime, "total minutes of parking"] += cur_minutesInHour
    
    return this_df

In [9]:
agg_rf1_data = agg_rows(dis_rf1_data, 2018, 2021)
agg_rf2_data = agg_rows(dis_rf2_data, 2018, 2021)

In [10]:
agg_rf1_data.head()

,datetime,hour,weekday,dayOfMonth,month,year,total number of sessions,total minutes of parking
2018-04-24 00:00:00+00:00,2018-04-24 00:00:00+00:00,0,1,24,4,2018,0,0.0
2018-04-24 01:00:00+00:00,2018-04-24 01:00:00+00:00,1,1,24,4,2018,0,0.0
2018-04-24 02:00:00+00:00,2018-04-24 02:00:00+00:00,2,1,24,4,2018,0,0.0
2018-04-24 03:00:00+00:00,2018-04-24 03:00:00+00:00,3,1,24,4,2018,0,0.0
2018-04-24 04:00:00+00:00,2018-04-24 04:00:00+00:00,4,1,24,4,2018,0,0.0


Now we import the hourly weather data

In [11]:
weatherData = pd.read_csv("cleanWeatherDataFeatureEngineering.csv")
weatherData["timestamp"] = pd.to_datetime(weatherData["timestamp"])
weatherData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      26304 non-null  datetime64[ns]
 1   temperature    26304 non-null  float64       
 2   cloud_cover    26304 non-null  float64       
 3   pressure       26304 non-null  float64       
 4   windspeed      26304 non-null  float64       
 5   precipitation  26304 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 1.2 MB


In [12]:
weatherData["timestamp"] = weatherData["timestamp"].dt.round("h")
weatherData = weatherData.rename(columns={"timestamp": "datetime"})
weatherData

,datetime,temperature,cloud_cover,pressure,windspeed,precipitation
0,2018-01-01 09:00:00,9.0,33.0,991.75,9.0,0.0
1,2018-01-01 10:00:00,9.0,33.0,992.08,0.0,0.0
2,2018-01-01 11:00:00,9.0,21.0,992.08,0.0,0.0
3,2018-01-01 12:00:00,9.0,29.0,992.08,0.0,0.0
4,2018-01-01 13:00:00,8.0,33.0,992.08,0.0,0.0
...,...,...,...,...,...,...
26299,2021-01-01 04:00:00,13.0,33.0,986.81,0.0,0.0
26300,2021-01-01 05:00:00,12.0,33.0,986.81,11.0,0.0
26301,2021-01-01 06:00:00,12.0,33.0,987.47,9.0,0.0
26302,2021-01-01 07:00:00,11.0,33.0,987.14,13.0,0.0


In [13]:
agg_rf1_data["datetime"]

2018-04-24 00:00:00+00:00   2018-04-24 00:00:00+00:00
2018-04-24 01:00:00+00:00   2018-04-24 01:00:00+00:00
2018-04-24 02:00:00+00:00   2018-04-24 02:00:00+00:00
2018-04-24 03:00:00+00:00   2018-04-24 03:00:00+00:00
2018-04-24 04:00:00+00:00   2018-04-24 04:00:00+00:00
                                       ...           
2021-09-14 19:00:00+00:00   2021-09-14 19:00:00+00:00
2021-09-14 20:00:00+00:00   2021-09-14 20:00:00+00:00
2021-09-14 21:00:00+00:00   2021-09-14 21:00:00+00:00
2021-09-14 22:00:00+00:00   2021-09-14 22:00:00+00:00
2021-09-14 23:00:00+00:00   2021-09-14 23:00:00+00:00
Freq: h, Name: datetime, Length: 29760, dtype: datetime64[ns, UTC]

In [14]:
agg_rf1_data["datetime"] = agg_rf1_data["datetime"].dt.tz_localize(None)
agg_rf2_data["datetime"] = agg_rf2_data["datetime"].dt.tz_localize(None)
agg_rf1_data.head()

,datetime,hour,weekday,dayOfMonth,month,year,total number of sessions,total minutes of parking
2018-04-24 00:00:00+00:00,2018-04-24 00:00:00,0,1,24,4,2018,0,0.0
2018-04-24 01:00:00+00:00,2018-04-24 01:00:00,1,1,24,4,2018,0,0.0
2018-04-24 02:00:00+00:00,2018-04-24 02:00:00,2,1,24,4,2018,0,0.0
2018-04-24 03:00:00+00:00,2018-04-24 03:00:00,3,1,24,4,2018,0,0.0
2018-04-24 04:00:00+00:00,2018-04-24 04:00:00,4,1,24,4,2018,0,0.0


We map the two dataframes together on timestamp and connectionTime with a RIGHT Join on connectionTime

In [15]:
final_rf1 = pd.merge(agg_rf1_data, weatherData, how="left", on="datetime")
final_rf2 = pd.merge(agg_rf2_data, weatherData, how="left", on="datetime")

In [16]:
final_rf1

,datetime,hour,weekday,dayOfMonth,month,year,total number of sessions,total minutes of parking,temperature,cloud_cover,pressure,windspeed,precipitation
0,2018-04-24 00:00:00,0,1,24,4,2018,0,0.0,24.0,30.0,985.82,19.0,0.0
1,2018-04-24 01:00:00,1,1,24,4,2018,0,0.0,25.0,30.0,985.49,17.0,0.0
2,2018-04-24 02:00:00,2,1,24,4,2018,0,0.0,20.0,30.0,985.82,15.0,0.0
3,2018-04-24 03:00:00,3,1,24,4,2018,0,0.0,17.0,29.0,986.48,17.0,0.0
4,2018-04-24 04:00:00,4,1,24,4,2018,0,0.0,16.0,29.0,987.14,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29755,2021-09-14 19:00:00,19,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN
29756,2021-09-14 20:00:00,20,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN
29757,2021-09-14 21:00:00,21,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN
29758,2021-09-14 22:00:00,22,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN


In [17]:
final_rf2

,datetime,hour,weekday,dayOfMonth,month,year,total number of sessions,total minutes of parking,temperature,cloud_cover,pressure,windspeed,precipitation
0,2018-04-24 00:00:00,0,1,24,4,2018,0,0.0,24.0,30.0,985.82,19.0,0.0
1,2018-04-24 01:00:00,1,1,24,4,2018,0,0.0,25.0,30.0,985.49,17.0,0.0
2,2018-04-24 02:00:00,2,1,24,4,2018,0,0.0,20.0,30.0,985.82,15.0,0.0
3,2018-04-24 03:00:00,3,1,24,4,2018,0,0.0,17.0,29.0,986.48,17.0,0.0
4,2018-04-24 04:00:00,4,1,24,4,2018,0,0.0,16.0,29.0,987.14,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29755,2021-09-14 19:00:00,19,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN
29756,2021-09-14 20:00:00,20,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN
29757,2021-09-14 21:00:00,21,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN
29758,2021-09-14 22:00:00,22,1,14,9,2021,0,0.0,NaN,NaN,NaN,NaN,NaN


After thinking about what data to take, we decided to cut the data off after 2020-03. Thats when Corona hit and for all dates after that, we have no or few data. This also solves the problem of not having enough weather data.

In [22]:
final_rf2 = final_rf2[final_rf2["datetime"] <= "2020-03-31 23:00:00"]
final_rf1 = final_rf1[final_rf1["datetime"] <= "2020-03-31 23:00:00"]
final_rf1

,datetime,hour,weekday,dayOfMonth,month,year,total number of sessions,total minutes of parking,temperature,cloud_cover,pressure,windspeed,precipitation
0,2018-04-24 00:00:00,0,1,24,4,2018,0,0.000000,24.0,30.0,985.82,19.0,0.0
1,2018-04-24 01:00:00,1,1,24,4,2018,0,0.000000,25.0,30.0,985.49,17.0,0.0
2,2018-04-24 02:00:00,2,1,24,4,2018,0,0.000000,20.0,30.0,985.82,15.0,0.0
3,2018-04-24 03:00:00,3,1,24,4,2018,0,0.000000,17.0,29.0,986.48,17.0,0.0
4,2018-04-24 04:00:00,4,1,24,4,2018,0,0.000000,16.0,29.0,987.14,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16987,2020-03-31 19:00:00,19,1,31,3,2020,5,300.000000,22.0,34.0,988.45,11.0,0.0
16988,2020-03-31 20:00:00,20,1,31,3,2020,5,300.000000,23.0,34.0,987.47,13.0,0.0
16989,2020-03-31 21:00:00,21,1,31,3,2020,5,300.000000,24.0,34.0,986.48,11.0,0.0
16990,2020-03-31 22:00:00,22,1,31,3,2020,5,268.916667,25.0,34.0,985.82,11.0,0.0


In [26]:
final_rf1.to_csv("FINAL1.csv", index=False)
final_rf2.to_csv("FINAL2.csv", index=False)

**Requirements input data set for task 4:**
- Rows need to be aggregated by hour;
- Following columns are needed
    - ID
    - connectionTime
    - disconnectTime
    - doneChargingTime

    Date information:
    - hour
    - weekday 
    - day of month
    - month
    - year

    Aggregated utilization information (**possible target variables**):
    - hourly parking utilization (sum of total minutes of utilization of every parking space in specific hour)
    - hourly charging utilization (sum of total minutes of utilization of every parking space in specific hour)
    - hourly non-charging utilization = hourly parking utilization - hourly charging utilization
    - total kWh requested (sum of total kWh requested in specific hour; only consider kWh of sessions STARTED in specific hour)
    - total number of sessions

    Weather information:
    - temperature
    - cloud cover 
    - precipitation